# Library Imports

In [46]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [2]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

'2.2.5'

In [24]:
class Metrics(Callback):
    def on_train_begin(self, logs={}):
         self.val_f1s = []
         self.val_recalls = []
         self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
         val_predict = (np.asarray(self.model.predict(self.model.validation_data[0]))).round()
         val_targ = self.model.validation_data[1]
         _val_f1 = f1_score(val_targ, val_predict)
         _val_recall = recall_score(val_targ, val_predict)
         _val_precision = precision_score(val_targ, val_predict)
         self.val_f1s.append(_val_f1)
         self.val_recalls.append(_val_recall)
         self.val_precisions.append(_val_precision)
         print(' — val_f1: {0:f} — val_precision: {1:f} — val_recall {2:f}'.format(_val_f1, _val_precision, _val_recall))
         return

metrics = Metrics()

In [3]:
os.chdir('/Users/sean/CloudStation/Metis/projects/project5')
!pwd

/Users/sean/CloudStation/Metis/projects/project5


# Load Data

In [4]:
# load data
raw_data = './data/raw/tox21/'
y_tr = pd.read_csv(raw_data+'tox21_labels_train.csv.gz', index_col=0, compression="gzip")
y_te = pd.read_csv(raw_data+'tox21_labels_test.csv.gz', index_col=0, compression="gzip")
x_tr_dense = pd.read_csv(raw_data+'tox21_dense_train.csv.gz', index_col=0, compression="gzip").values
x_te_dense = pd.read_csv(raw_data+'tox21_dense_test.csv.gz', index_col=0, compression="gzip").values
x_tr_sparse = io.mmread(raw_data+'tox21_sparse_train.mtx.gz').tocsc()
x_te_sparse = io.mmread(raw_data+'tox21_sparse_test.mtx.gz').tocsc()
# filter out very sparse features
sparse_col_idx = ((x_tr_sparse > 0).mean(0) > 0.05).A.ravel()
x_tr = np.hstack([x_tr_dense, x_tr_sparse[:, sparse_col_idx].A])
x_te = np.hstack([x_te_dense, x_te_sparse[:, sparse_col_idx].A])

# Choose A Target

In [5]:
y_tr.columns

Index(['NR.AhR', 'NR.AR', 'NR.AR.LBD', 'NR.Aromatase', 'NR.ER', 'NR.ER.LBD',
       'NR.PPAR.gamma', 'SR.ARE', 'SR.ATAD5', 'SR.HSE', 'SR.MMP', 'SR.p53'],
      dtype='object')

The Random Forest example loops through all the targets.  I'll pick only the first one for the DNN MVP:

In [6]:
# for target in y_tr.columns:
target = 'NR.AhR'
rows_tr = np.isfinite(y_tr[target]).values
rows_te = np.isfinite(y_te[target]).values
x,y = x_tr[rows_tr], y_tr[target][rows_tr]
x.shape

(8441, 1644)

# Address Class Imbalance
Oversampling Documentation:
https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html

"While the `RandomOverSampler` is over-sampling by duplicating some of the original samples of the minority class, `SMOTE` and `ADASYN` generate new samples in by interpolation. However, the samples used to interpolate/generate new synthetic samples differ. In fact, `ADASYN` focuses on generating samples next to the original samples which are wrongly classified using a k-Nearest Neighbors classifier while the basic implementation of `SMOTE` will not make any distinction between easy and hard samples to be classified using the nearest neighbors rule. Therefore, the decision function found during training will be different among the algorithms."

**I decided that over-sampling using synthetic methods is probably not legitimate because it is creating new "samples", i.e. chemicals with properties (feature values) that do not represent real chemical structures.  Though I tried using SMOTE and got reasonably similar results, I think the approach is technically dubious.**

In [7]:
from imblearn.over_sampling import RandomOverSampler #, SMOTE, ADASYN

In [8]:
y.value_counts()

0.0    7460
1.0     981
Name: NR.AhR, dtype: int64

To keep the class proportions the same use the stratify parameter: [source](https://stats.stackexchange.com/questions/394056/splitting-into-train-and-test-sets-keeping-class-proportions)

In [9]:
x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

In [10]:
y_val.value_counts()

0.0    1493
1.0     196
Name: NR.AhR, dtype: int64

In [11]:
ros = RandomOverSampler(random_state=0)
# ros = SMOTE(random_state=42)   # See comment above - I don't believe this is legitimate.
x_resampled, y_resampled = ros.fit_sample(x_train,y_train)

In [12]:
pd.Series(y_resampled).value_counts()

1.0    5967
0.0    5967
dtype: int64

In [13]:
x_resampled.shape

(11934, 1644)

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [51]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
#     return recall_score(y_true.eval(),y_pred.eval())

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [52]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential([
    keras.layers.InputLayer(input_shape=x.shape[1:],name='Input_Layer')
])
for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h1_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h3_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout3 (Dropout)           (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [38]:
DNN.fit(
    x_resampled, y_resampled, batch_size=512, epochs=100,\
    validation_data=(x_val,y_val), verbose=1,
    callbacks=[
        keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                      restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
    ])

Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 2s 184us/step - loss: 0.6167 - acc: 0.7585 - recall_m: 0.7984 - val_loss: 0.6145 - val_acc: 0.7863 - val_recall_m: 0.7273
Epoch 2/100
11934/11934 [==============================] - 2s 191us/step - loss: 0.6193 - acc: 0.7567 - recall_m: 0.7973 - val_loss: 0.6140 - val_acc: 0.7880 - val_recall_m: 0.7273
Epoch 3/100
11934/11934 [==============================] - 2s 197us/step - loss: 0.6191 - acc: 0.7592 - recall_m: 0.7916 - val_loss: 0.6139 - val_acc: 0.7886 - val_recall_m: 0.7273
Epoch 4/100
11934/11934 [==============================] - 3s 213us/step - loss: 0.6182 - acc: 0.7617 - recall_m: 0.8044 - val_loss: 0.6138 - val_acc: 0.7874 - val_recall_m: 0.7273
Epoch 5/100
11934/11934 [==============================] - 2s 209us/step - loss: 0.6192 - acc: 0.7547 - recall_m: 0.7921 - val_loss: 0.6136 - val_acc: 0.7874 - val_recall_m: 0.7273
Epoch 6/100
11934/11934 [=====================

In [40]:
auc_te = roc_auc_score(y_te[target][rows_te], DNN.predict(x_te[rows_te]))
print("%15s: %3.5f" % (target, auc_te))

         NR.AhR: 0.86169


In [41]:
y_testing=y_te[target][~np.isnan(y_te[target])]
y_hat_testing=DNN.predict_classes(x_te[rows_te])
print(np.array([['TN','FP'],['FN','TP']]))
print(confusion_matrix(y_testing,y_hat_testing))

[['TN' 'FP']
 ['FN' 'TP']]
[[401 136]
 [ 13  60]]


In [42]:
print('f1:',f1_score(y_testing,y_hat_testing))
print('recall:',recall_score(y_testing,y_hat_testing))
print('precision:',precision_score(y_testing,y_hat_testing))

f1: 0.44609665427509304
recall: 0.821917808219178
precision: 0.30612244897959184


In [19]:
y_te[target][rows_te].value_counts()

0.0    537
1.0     73
Name: NR.AhR, dtype: int64

In [ ]:
537/(537+73)

Uncomment to save model.  Last ROC_UAC = 0.86068

In [43]:
# DNN.save('./models/saves/first_model.h5')